In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np
import json
import tensorflow_text as tf_text
import FTokenizer
import random
import wikipedia as wiki

In [2]:
# Open data Intent,Bi_gram,VN Stop word,posData

with open("C:/Projects/personalProjects/pythonStuff/foxPython/ChatBot/intentsVNver1.json",encoding="utf-8") as file:
    data=json.load(file)
patternlist=[]
labels=[]
for dt in data["intents"]:
    for pattern in dt["patterns"]:
        patternlist.append(pattern)
        labels.append(dt["tag"])
labels
labels=np.array(labels)
labelsOH=pd.get_dummies(labels)
labellist=list(labelsOH)

with open("C:/Projects/personalProjects/pythonStuff/foxPython/data/bi_grams.txt",encoding="utf-8") as f:
    bi_gramtxt=f.read()
bi_gram_list=[]
for each in bi_gramtxt.split(","):
    each=each[:-1]
    each=each[2:]
    each=each.replace(" ","_")
    bi_gram_list.append(each)

with open("C:/Projects/personalProjects/pythonStuff/foxPython/data/vietnamese-stopwords.txt",encoding="utf-8") as f:
    stopword_txt=f.readlines()
    
stopword_list=[]
for each in stopword_txt:
    stopword_list.append(each[:-1])

with open("C:/Projects/personalProjects/pythonStuff/foxPython/NewTagger/posdata.json","r",encoding="utf-8") as f:
    mpos=json.load(f)

responses_dict={}
for i in data["intents"]:
    responses_dict[i["tag"]]=i["responses"]


In [7]:
#Class tạo tách từ 
class FTokenizer():
    def __init__(self,bi_gram_list):
        self.bi_gram_list=bi_gram_list

    def bi_gram_checker(self,bi_gram_word):
        for i in self.bi_gram_list:
            if (bi_gram_word==i):
                return True
        return False
    
    def fit_on_text(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (myContinue):
                    myContinue=False
                    continue
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        myContinue=True
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (myContinue==False):  
                    if (tokens.count(listtokens[i])==0):
                        tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word

    def fit_on_text2(self,data):
        word_to_index={}
        index_to_word={}
        word_to_index["<PAD>"]=0
        word_to_index["<OOV>"]=1
        index_to_word[0]="<PAD>"
        index_to_word[1]="<OOV>"
        myContinue=False
        tokens=[]
        for eachseq in data:
            listtokens=eachseq.lower().split(" ")
            listtokens.append("")
            for i in range(0,len(listtokens)-1):
                if (listtokens[i+1]!=""):
                    pairtext=listtokens[i]+"_"+listtokens[i+1]
                    if (self.bi_gram_checker(pairtext)):
                        if (tokens.count(pairtext)==0):
                            tokens.append(pairtext)
                if (tokens.count(listtokens[i])==0 and listtokens[i]!=""):
                    tokens.append(listtokens[i])
        for index,word in enumerate(tokens):
            word_to_index[word]=index+2
            index_to_word[index+2]=word
        self.word_to_index=word_to_index
        self.index_to_word=index_to_word
    
    def get_word_to_index(self):
        return self.word_to_index
    
    def get_index_to_word(self):
        return self.index_to_word

    def get_max_length(self,data):
        maxsen=0
        for i in data:
            if maxsen<len(i):
                maxsen=len(i)
        return maxsen

    def text_to_sequence(self,textdata):
        d_continue=False
        output=[]
        for text in textdata:
            text=text.lower().split(" ")
            text.append("")
            sth=[]
            d_continue=False
            for i in range(0,len(text)-1):
                if d_continue==True:
                    d_continue=False
                    continue
                if (text[i+1]!=""):
                    pairtext=text[i]+"_"+text[i+1]
                    for j in self.word_to_index:
                        if pairtext.lower()==j:
                            sth.append(self.word_to_index[j])
                            d_continue=True
                            break
                if d_continue==False:
                    for count,j in enumerate(self.word_to_index):
                        if text[i].lower()==j:
                            sth.append(self.word_to_index[j])
                            break
                        if count==len(self.word_to_index)-1:
                            sth.append(1)
            output.append(sth)
        return output
    
    def intseq_to_index(self,sequence):
        output=[]
        for i in sequence:
            output.append(self.index_to_word[i])
        return output

    def pad_sequence(self,datasequence,maxlen):
        for i in datasequence:
            while len(i)!=maxlen:
                i.insert(0,0)
        return datasequence

    def bi_gram_ize(self,text):
        listtokens=text.lower().split(" ")
        listtokens.append("")
        myContinue=False
        tokens=[]
        for i in range(0,len(listtokens)-1):
            if (myContinue):
                myContinue=False
                continue
            if (listtokens[i+1]!=""):
                pairtext=listtokens[i]+"_"+listtokens[i+1]
                if (self.bi_gram_checker(pairtext)):
                    myContinue=True
                    if (tokens.count(pairtext)==0):
                        tokens.append(pairtext)
            if (myContinue==False):  
                if (tokens.count(listtokens[i])==0):
                    tokens.append(listtokens[i])
        return tokens


In [4]:
# VNTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,oov_token="<UNK>")
# VNTokenizer.fit_on_texts(patternlist)
# pattern_tokenized=VNTokenizer.texts_to_sequences(patternlist)
# pattern_preprocessed=tf.keras.preprocessing.sequence.pad_sequences(pattern_tokenized)
# mlen=pattern_preprocessed.shape[1]
# tokens_size=len(VNTokenizer.word_index)

In [8]:
#Apply Model tách từ vào các data intent,POS

myTokenizer=FTokenizer(bi_gram_list)
myTokenizer.fit_on_text2(patternlist)
pattern_tokenized2=myTokenizer.text_to_sequence(patternlist)
pattern_preprocessed2=tf.keras.preprocessing.sequence.pad_sequences(pattern_tokenized2)
mlen2=pattern_preprocessed2.shape[1]

posmesTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,filters='!"#$%&()*+,-./:;<=>?@[\\]^`{|}~\t\n',oov_token="<UNK>")
posmesTokenizer.fit_on_texts(mpos["messages"])
mes_Tokenized=posmesTokenizer.texts_to_sequences(mpos["messages"])
finalmes=tf.keras.preprocessing.sequence.pad_sequences(mes_Tokenized)

postagTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000)
postagTokenizer.fit_on_texts(mpos["pos"])
pos_tokenized=postagTokenizer.texts_to_sequences(mpos["pos"])
finalpos=tf.keras.preprocessing.sequence.pad_sequences(pos_tokenized)

In [6]:
#Load weight cho Embedding
with open("D:/a-Code/Ignite Niek/foxPython/word_2_vec/data/new_vocabs_weights4.json","r",encoding="utf-8") as f:
    vocab_weights=json.load(f)
len(vocab_weights)

7877

In [7]:
#Đưa weight vào Embedding layer
num_tokens = len(myTokenizer.word_to_index)
embedding_dim = 128
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in myTokenizer.word_to_index.items():
    embedding_vector = vocab_weights.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

embedding_layer = tf.keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Converted 187 words (14 misses)


In [9]:
def LSTMModel():
    int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
    # x=embedding_layer(int_sequences_input)
    x=tf.keras.layers.Embedding(len(myTokenizer.word_to_index),128)(int_sequences_input)
    x=tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64,return_sequences=True))(x)
    x=tf.keras.layers.GlobalMaxPool1D()(x)
    x=tf.keras.layers.Dense(16,activation="relu")(x)
    x=tf.keras.layers.Dense(len(labellist),activation="softmax")(x)
    return tf.keras.Model(int_sequences_input,x)
    

In [10]:
model=LSTMModel()
model.compile(tf.keras.optimizers.Adam(learning_rate=0.02),loss="categorical_crossentropy",metrics="categorical_accuracy")

In [11]:
model.fit(pattern_preprocessed2,labelsOH,batch_size=16,epochs=250)

Epoch 1/250
8/8 [==============================] - 7s 9ms/step - loss: 2.6389 - categorical_accuracy: 0.0603
Epoch 2/250
8/8 [==============================] - 0s 6ms/step - loss: 2.3794 - categorical_accuracy: 0.1983
Epoch 3/250
8/8 [==============================] - 0s 6ms/step - loss: 1.7802 - categorical_accuracy: 0.4310
Epoch 4/250
8/8 [==============================] - 0s 7ms/step - loss: 0.8546 - categorical_accuracy: 0.7672
Epoch 5/250
8/8 [==============================] - 0s 6ms/step - loss: 0.4752 - categorical_accuracy: 0.8621
Epoch 6/250
8/8 [==============================] - 0s 6ms/step - loss: 0.1968 - categorical_accuracy: 0.9569
Epoch 7/250
8/8 [==============================] - 0s 6ms/step - loss: 0.0778 - categorical_accuracy: 0.9914
Epoch 8/250
8/8 [==============================] - 0s 6ms/step - loss: 0.0651 - categorical_accuracy: 0.9828
Epoch 9/250
8/8 [==============================] - 0s 7ms/step - loss: 0.0245 - categorical_accuracy: 1.0000
Epoch 10/250
8/8 [=

In [11]:
#tagModel=tf.keras.models.load_model("E:/MLCourse/TF2/Notebook/SmartFennec/NER file/TagModel")
POStagger=tf.keras.models.load_model("D:/a-Code/Ignite Niek/foxPython/NewTagger/PosTaggerModel")

In [12]:
# with open("E:/MLCourse/TF2/Notebook/SmartFennec/NER file/tagtrain.json","r",encoding="utf-8") as f:
#     data=json.load(f)
# textlist=[]

# for each in data:
#     textlist.append(each["sentence"])

# TextTokenizer=tf.keras.preprocessing.text.Tokenizer(num_words=20000,lower=True,oov_token="<UNK>")
# TextTokenizer.fit_on_texts(textlist)

# tokenizedText=TextTokenizer.texts_to_sequences(textlist)
# textFinal=tf.keras.preprocessing.sequence.pad_sequences(tokenizedText)

# def insert_text(text):
#     text_tokenized=TextTokenizer.texts_to_sequences([text])
#     finaltext=tf.keras.preprocessing.sequence.pad_sequences(text_tokenized,maxlen=textFinal.shape[1])
#     return finaltext


In [13]:
# def entitiesOutput(rs):
#     txt=[]
#     for i in rs[0]:
#         txt.append(tagdict[np.argmax(i)])
#         # use argmax 
#     return txt

In [14]:
#Mình sẽ đưa mấy hoạt động được kích hoạt ở đây
def action_activate(input,entities):
    if input=="question_from_user":
        if len(entities)!=0:
            noun=pos_filter(entities,"noun")
            s=""
            for i in noun:
                s=s+i[0].replace("_"," ")+" "
            return wiki.summary(s)
        return "Sorry, hiện tại mình chưa biết nữa"

    if input=="askingme":
        noun=pos_filter(entities,"noun")
        verb=pos_filter(entities,"verb")
        known_list=["sách","Toán","Văn","Anh","Minecraft","LOL"]
        for i in known_list:
            if noun[0].count(i.lower())!=0:
                return "Yess"
        return "Nah"
    return "//"

In [15]:
# Memory cho Bot(tạm thời)
class ShortTermMemory():
    def __init__(self):
        self.mem={}
        self.dialogue_count=0
        self.length_limit=20
    
    def reset(self):
        self.mem={}
    
    def add(self,intent,sentence_entities):
        self.mem[self.dialogue_count]=[intent,sentence_entities]
        self.dialogue_count=self.dialogue_count+1
        if self.dialogue_count==self.length_limit:
            self.mem={}

    def lookup(self):
        return self.mem

In [16]:
#Chấp nhận acc trên n% 
def accept_accuracy(pred,acc):
    if (max(pred)>acc):
        return np.argmax(pred)
    else:
        return len(pred)
#Enable PrintMode
def debugMode(printlist):
    for count,i in enumerate(printlist):
        print(count,"/",i)

In [17]:
#Convert từ text qua số
posmesTokenizer.index_word[0]="<PAD>"
postagTokenizer.index_word[0]="<PAD>"
def insert_text(inp):
    inp_gramized=myTokenizer.bi_gram_ize(inp)
    inp_tokenized=posmesTokenizer.texts_to_sequences([inp_gramized])
    final_inp=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=finalmes.shape[1])
    return final_inp
#Xử lý output data, biến thành dạng [Từ,POS]
def posAnalyze(inp,posdata):
    pos=[]
    for i in posdata[0]:
        if postagTokenizer.index_word[np.argmax(i)]!="<PAD>":
            pos.append(postagTokenizer.index_word[np.argmax(i)])
    l=[]
    for i in range(0,len(pos)):
        if pos[i]!="<PAD>":
            l.append([inp[i],pos[i]])
    return l
#Filter loại từ
def pos_filter(pos_rs,pos_name,stopword=["bạn","tớ","cậu","tí","này","lát","chút","nhé","nha","mình"]):
    pos_filtered=[]
    for i in pos_rs:
        if (i[1]==pos_name and stopword.count(i[0])==0):
            pos_filtered.append(i)
    return pos_filtered

In [18]:
#Set up
import firebase_admin
from firebase_admin import credentials
from firebase_admin import firestore
from random import randint
cred = credentials.Certificate("D:/a-Code/Ignite Niek/foxPython/ChatBot/android-firebase-1f1b8-firebase-adminsdk-wtlm2-7a0d32b4fc.json")
firebase_admin.initialize_app(cred)
db = firestore.client()

In [19]:
def checker(sentence):
    #Example format
    def sentences(sub,item):
        sen1 = [f"Bạn đã thử {sub} {item} chưa?",
        f"Mình thấy {sub} {item} cũng được lắm nha!",
        f"Nhiều người thích {sub} {item} lắm đó!"]
        return sen1[randint(0,2)]
    #Things that db contains
    names = []
    docs = db.collection(u'bot').stream()
    for doc in docs:
        names.append(doc.id)
    #Check if the asked thing in list
    for word in sentence.split():
        if word in names:
            doc_ref = db.collection(u'bot').document(word)
            doc = doc_ref.get()
            if doc.exists:
                dictz = doc.to_dict()
                return(sentences(word,dictz["item"][randint(0,len(dictz["item"])-1)]))
                break

In [21]:
postagTokenizer.index_word[0]="<PAD>"
wiki.set_lang("vi")
labelsre=np.array(list(labelsOH.columns)+["NULL"])
def chat():
    memory=ShortTermMemory()
    print("start")
    while True:
        reply=""
        inp=input("You: ")
        if inp=="quit":
            break
        inp_tokenized=myTokenizer.text_to_sequence([inp])
        inp_preprocessed=tf.keras.preprocessing.sequence.pad_sequences(inp_tokenized,maxlen=mlen2)
        pos_pred=POStagger.predict(insert_text(inp))
        pos_rs=posAnalyze(myTokenizer.bi_gram_ize(inp),pos_pred)
        rs=np.round(model.predict(inp_preprocessed),decimals=2)
        pred=labelsre[accept_accuracy(rs[0],0.8)]
        print("User: ",inp)
        #debugMode([inp,myTokenizer.intseq_to_index(inp_preprocessed[0]),pos_rs,pred,pos_filter(pos_rs,"noun"),pos_filter(pos_rs,"verb")])
        memory.add(pred,pos_rs)
        if (pred!="askingme" and pred!="NULL"):
            reply=random.choice(responses_dict[pred])
        if pred=="NULL":
            reply="Là sao vậy ? Mình không hiểu cậu lắm !"
        print("Fennec: ",reply)
        print("Fennec db : ",checker(inp))
        print(action_activate(pred,pos_rs))
        print("")
    print("this conversation memory: ",memory.lookup())
chat()

start
User:  hi banj
Fennec:  Là sao vậy ? Mình không hiểu cậu lắm !
Fennec db :  None
//

User:  bạn thích coi phim gì
Fennec:  Check it out
Mình thấy phim Parasite cũng được lắm nha!
Fennec db :  None


D:\anaconda3\lib\site-packages\wikipedia\wikipedia.py:389: GuessedAtParserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 389 of the file D:\anaconda3\lib\site-packages\wikipedia\wikipedia.py. To get rid of this warning, pass the additional argument 'features="lxml"' to the BeautifulSoup constructor.

  lis = BeautifulSoup(html).find_all('li')


DisambiguationError: "Phim (định hướng)" may refer to: 
Điện ảnh
Phim truyền hình
Phim chiếu mạng
Phim điện ảnh
Điện ảnh truyền hình
Phim video
Phim băng đĩa
Phim nhựa
Phim hoạt hình
Phim tình cảm
Rạp chiếu phim
Kỹ thuật điện ảnh
Quá trình làm phim
^

issue: (1)khi được hỏi, bot sẽ cư xử sao ?
        (2) Khi nào mình sẽ đưa data vào brain để sử dụng sau
        (3) Structure trả lời sẽ như thế nào để phù hợp với (1)
        (4) Limit của cấu trúc này là bot sẽ rất bị động
        (5) intents sao mà Bot đạt tối ưu nhất có thể
        (6) Nếu entities liên kết với nhau thì mình sẽ có gì
        (7) Bot decide được không

(1)Solution:
Mỗi situation sẽ có một dictionary để fully kích hoạt
{asking -> {@chủ_đề->[@main, @vấn_đề}]}
Ví dụ: {askingme -> {studying->[môn học, vấn đề được hỏi}]}

(2)Tạm thời là mọi thông tin (dưới dạng dictionary), để search thông tin thì mình sẽ đo sự tương quan giữa hai câu (có thể từ entities)

(3) có thể sẽ liên quan đến data của (5)

(4) có thể random hỏi user nếu user có đặc điểm nào đó đáng lưu ý

(7) Environment: Chat dialogue ,Action: Chat với người dùng, Reward: user's sentiment